======================== Import Packages ==========================

In [12]:
import sys,os,pdb,glob
import numpy as np
from astropy import constants as const
from astropy import units as u
from astropy.table import Table, join, MaskedColumn
from astroquery.vizier import Vizier
import warnings
from astropy.logger import AstropyWarning
warnings.filterwarnings('ignore', category=AstropyWarning)

===================== Define Functions ===================

In [2]:
def get_data(catalog, join_key='Name', join_type='inner'):

    """
    PURPOSE:    Get data from literature with Vizier

    INPUT:      catalog = ctalog name on Vizier (str)
                join_key = column header to join tables, if multiple (str; optional)
                join_type = way to join tables, if multiple (str; optional)

    OUTPUT:     t = data table (AstroPy Table)

    """

    ### GET FULL CATALOG (ALL COLUMNS, ALL ROWS)
    viz = Vizier(catalog=catalog, columns=['**'])
    viz.ROW_LIMIT = -1
    tv = viz.get_catalogs(catalog)

    ### IF MULTIPLE TABLES, JOIN THEN
    for i, val in enumerate(tv.keys()):
        if i == 0:
            t = tv[val]
        else:
            tt = tv[val]
            if join_key in tt.columns:
                t = join(t, tt, join_type=join_type, keys=join_key)

    return t

In [3]:
def fix_units(b, l, f, d, t):
    
    """
    PURPOSE:    Change units of inputs to work with
                dust mass calculations

    INPUT:      b = dust opacity power-law index (unitless; float)
                l = wavelength of observations (microns; float)
                f = observed flux (mJy; float)
                d = istance to disk (pc; float)
                t = isk temperature (K; float)

    OUTPUT:     Inputs but now with correct units

    """

    b = float(b) 
    t = float(t) * u.K 
    d = float(d) * u.pc.to(u.cm) * u.cm
    l = float(l) * u.um.to(u.cm) * u.cm 
    f = float(f) * u.mJy

    return b, l, f, d, t

In [4]:
def get_planck(l, t):

    """
    PURPOSE:    Calculate Planck function

    INPUT:      l = wavelength of observations in cm (float)
                t = disk temperature in K (float)

    OUTPUT:     planck = Planck function (float)

    """

    ### CONVERT WAVELENGTH TO FREQUENCY
    nu = (const.c.cgs / l)

    ### CALCULATE PLANCK FUNCTION
    plank = 2 * const.h.cgs * nu**3 / const.c.cgs**2 / (np.exp(const.h.cgs *nu / (const.k_B.cgs * t)) -1)

    return plank

In [5]:
def get_kappa(l, b):

    """
    PURPOSE:    Calculate kappa parameter for dust mass calculation

    INPUT:      l = wavelength of observations in cm (float)
                b = dust opacity power-law index (float)

    OUTPUT:     kappa = kappa parameter for dust mass calculation (float)

    """

    ### CONVERT WAVELENGTH TO FREQUENCY
    nu = (const.c.cgs / l)

    ### ASSUMES NORMALIZATION OF 2.3 cm^2/g at 230 GHz
    nf = 2.3 / ( (230. / 1000.)**b )

    ### CALCULATE KAPPA PARAMETER
    kappa = (nf * (nu / (1e9 * 1000 * u.Hz))**b ) * ((u.cm**2)/(u.g))
        
    return kappa 

In [6]:
def get_mult(d, kappa, b_nu):

    """
    PURPOSE:    Calculate multiplication factor that produces dust mass
                when applied to source flux in mJy
                (matches Eq. 1 in Ansdell+2016 when d=150pc)

    INPUT:      d = distance to source in cm (float)
                kappa = kappa parameter from get_kappa() (float)
                b_nu = Planck function from get_planck() (float)

    OUTPUT:     mult = mltiplication factor that produces dust mass
                       when applied to source flux in mJy (float)

    """

    mult = (1e-26) * (d**2) / (kappa * b_nu) / const.M_sun.cgs
        
    return mult

In [7]:
def get_dustmass(b, l, f, d, t):
    
    """
    PURPOSE:    Calculate disk dust mass using prescription from
                Hildebrand 1983 (1983QJRAS..24..267H)

    INPUT:      b = dust opacity power-law index (unitless; float)
                l = wavelength of observations (microns; float)
                f = observed flux (mJy; float)
                d = distance to disk (pc; float)
                t = disk temperature (K; float)

    OUTPUT:     Disk dust mass in Earth masses (float)

    """

    b, l, f, d, t = fix_units(b, l, f, d, t)
    k_nu = get_kappa(l, b)
    b_nu = get_planck(l, t)
    mult = get_mult(d, k_nu, b_nu)
    mdust = mult * f * (const.M_sun.cgs / const.M_earth.cgs)

    return round(mdust.value, 5)

============================= Code ==================================

In [8]:
#### LOAD IN LUPUS DATA
T = get_data("J/ApJ/828/46")

In [9]:
### GET DUST MASSES
DM, e_DM = [], []
for i, val in enumerate(T['Name']):

    ### SET INPUTS TO DUST MASS CALCULATION
    Beta_Index = 1.                   # DUST OPACITY POWER-LAW INDEX (UNITLESS)
    Lambda_Obs = 890.                 # WAVELENGTH OF OBSERVATIONS (MICRONS)
    Temp_Disk = 20.                   # DISK TEMPERATURE (KELVIN)
    Dist_Disk = T['Dist'][i]           # DISTANCE TO DISK (PC)
    Flux_Disk = T['F890'][i]         # OBSERVED FLUX (mJy)
    e_Flux_Disk = T['e_F890'][i]     # ERROR ON OBSERVED FLUX (mJy)

    ### CALCULATE DUST MASS
    DM.append(get_dustmass(Beta_Index, Lambda_Obs, Flux_Disk, Dist_Disk, Temp_Disk))
    e_DM.append(get_dustmass(Beta_Index, Lambda_Obs, e_Flux_Disk, Dist_Disk, Temp_Disk))

In [14]:
TD = Table()
TD['Name'] = np.copy(T['Name'])
TD.add_column(MaskedColumn(name='MDust', data=DM))
TD.add_column(MaskedColumn(name='e_MDust', data=e_DM))
display(TD)

Name,MDust,e_MDust
str17,float64,float64
J15430131-3409153,0.00235,0.07288
J15430227-3444059,0.05172,0.06347
J15445789-3423392,-0.01175,0.04232
J15450634-3417378,3.52637,0.09404
J15450887-3417333,10.87769,0.11755
J15592523-4235066,-0.00705,0.04467
J16000060-4221567,0.56422,0.04467
J16000236-4222145,28.17573,0.14811
J16002612-4153553,0.28211,0.04467


In [ ]:
### UNCOMMENT TO WRITE FILE 
# TD.write('../output/dustmasses.txt', format='ascii.ipac') 